In [62]:
import toml

import geopandas as gpd

In [63]:
df = gpd.read_file("./fuas/cities_fua.gpkg")
df = df.sort_values("FUA_area", ascending=False)
df = df[(df["country"] != "Argentina") & (df["country"] != "Barbados")]

In [64]:
class Bucket:
    def __init__(self):
        self.indices = []
        self.areas = []
        self.total_area = 0

    def add(self, new):
        self.indices.append(new[0])
        self.areas.append(new[1])
        self.total_area += new[1]

    def __repr__(self):
        return f"Indices: {self.indices}\nAreas: {self.areas}\nTotal: {self.total_area}"
    
    def __lt__(self, other):
        return self.total_area < other.total_area
    
    def __le__(self, other):
        return self.total_area <= other.total_area
    
    def __gt__(self, other):
        return self.total_area > other.total_area
    
    def __ge__(self, other):
        return self.total_area >= other.total_area

In [65]:
num_buckets = 10
buckets = []
for i in range(num_buckets):
    bucket = Bucket()
    index = int(df.index[i])
    area = df.iloc[i]["FUA_area"]
    bucket.add((index, area))
    buckets.append(bucket)

for index, row in df.iloc[num_buckets:].iterrows():
    buckets.sort()
    index = int(index)
    area = row["FUA_area"]
    buckets[0].add((index, area))

In [66]:
indices = [b.indices for b in buckets]
indices = [x for subl in indices for x in subl]
indices = set(indices)

In [67]:
with open("./config_master.toml", "r") as f:
    config_master = toml.load(f)

for i in range(num_buckets):
    config = config_master.copy()
    config["calibration"]["indices"] = buckets[i].indices
    with open(f"./configs/config_{i}.toml", "w") as f:
        toml.dump(config, f)